## Chapter 3 Problems
This notebook attempts to solve the problems in Chapter 3 of "Aurélien Géron - Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow_ Concepts, Tools, and Techniques to Build Intelligent Systems-O'Reilly Media (2022)".

In [25]:
#Import dependencies

import pandas as pd
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

## Question 1

1. Try to build a classifier for the MNIST dataset that achieves over 97% accuracy on the test set. Hint: the KNeighborsClassifier works quite well for this task; you just need to find good hyperparameter values (try a grid search on the weights and n_neighbors hyperparameters).

### Create training and test sets

In [2]:
mnist = fetch_openml('mnist_784', as_frame=False) 
X, y = mnist.data, mnist.target
some_input = X[0] #this is a 5
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

/Users/Cheks/opt/anaconda3/envs/python/lib/python3.9/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


### Scale and create classifier
train a KNeighbours classifier using a grid search to find the best hyperparameters

In [30]:

knn_clf = KNeighborsClassifier() #weights and n_neighbours hyperparameters

param_grid = [{'n_neighbors': [3,4,5,6],
               'weights': ['distance']}]

In [33]:
grid_search = GridSearchCV(knn_clf, 
                           param_grid= param_grid,
                           cv = 3,
                           scoring= 'accuracy'
)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': [3, 4, 5, 6, 7, 8],
                          'weights': ['distance']}],
             scoring='accuracy')

In [34]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.888161,0.727335,13.840598,0.760489,3,distance,"{'n_neighbors': 3, 'weights': 'distance'}",0.97040,0.96890,0.96870,0.969333,0.000759,3
1,0.343604,0.008568,11.410576,0.381231,4,distance,"{'n_neighbors': 4, 'weights': 'distance'}",0.97090,0.96980,0.97035,0.970350,0.000449,1
2,0.333869,0.001568,11.301501,0.267968,5,distance,"{'n_neighbors': 5, 'weights': 'distance'}",0.96880,0.96795,0.96905,0.968600,0.000471,4
3,0.333713,0.002992,11.784688,0.821179,6,distance,"{'n_neighbors': 6, 'weights': 'distance'}",0.97005,0.96855,0.96985,0.969483,0.000665,2
4,0.469934,0.128603,13.039963,1.286861,7,distance,"{'n_neighbors': 7, 'weights': 'distance'}",0.96700,0.96560,0.96740,0.966667,0.000772,6
5,0.357462,0.026740,15.740887,0.921795,8,distance,"{'n_neighbors': 8, 'weights': 'distance'}",0.96715,0.96620,0.96845,0.967267,0.000922,5


In [37]:
grid_search.score(X_test, y_test) #use .score to return the score using the given data when the best estimator has already been found

0.9714

## Question 2 - Data Augmentation

Write a function that can shift an MNIST image in any direction (left, right, up, or down) by one pixel.⁠ Then, for each image in the training set, create four shifted copies (one per direction) and add them to the training set. Finally, train your best model on this expanded training set and measure its accuracy on the test set. You should observe that your model performs even better now! This technique of artificially growing the training set is called data augmentation or training set expansion.